In [2]:
import pandas as pd
import os

In [3]:
df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(),'..','Data','cleaned_train.csv')))

In [4]:
df.head()

,id,activity_0_cnt,activity_1_cnt,activity_2_cnt,activity_3_cnt,activity_4_cnt,text_change_0_cnt,text_change_1_cnt,text_change_2_cnt,text_change_3_cnt,...,paragraph_word_count_max,paragraph_word_count_first,paragraph_word_count_last,paragraph_word_count_q1,paragraph_word_count_median,paragraph_word_count_q3,paragraph_word_count_sum,keys_per_second,product_to_keys,score
0,001519c8,2010,417,120,7,0,1940,436,28,14,...,112,71,86,78.50,86.0,99.00,269,1.350251,0.629584,3.5
1,0022f953,1938,260,254,1,1,1698,432,18,24,...,96,53,60,47.75,56.5,62.25,355,1.250038,0.762056,3.5
2,0042269b,3515,439,175,7,0,3257,615,23,26,...,88,79,45,55.50,73.5,78.75,410,2.237402,0.654274,6.0
3,0059420b,1304,151,99,1,1,1146,281,13,3,...,81,62,65,63.50,65.0,73.00,208,1.067440,0.793127,2.0
4,0075873a,1942,517,72,0,0,1964,397,32,25,...,114,61,3,26.00,52.0,61.00,256,1.552397,0.579504,4.0


In [6]:
from sklearn.model_selection import train_test_split,cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

#RFE and PCA for feature selection
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor  # Change from Classifier to Regressor
from catboost import CatBoostRegressor  # Change from Classifier to Regressor
from lightgbm import LGBMRegressor  # Change from Classifier to Regressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [7]:
ada_best = AdaBoostRegressor(n_estimators=100)
gb_best = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)
xgb_best = XGBRegressor(n_estimators=100, learning_rate=0.1)
lr_best = LinearRegression()
cat_best = CatBoostRegressor(iterations=100, learning_rate=0.1, verbose=0)
lgbm_best = LGBMRegressor(n_estimators=100, learning_rate=0.1)
svr_regressor = SVR(kernel='linear')
rforest_best = RandomForestRegressor(max_depth=10, n_estimators=150, random_state=42)

In [11]:
def handle_null(df):
    missing_columns = df.columns[df.isna().sum() > 0]
    df[missing_columns] = df[missing_columns].fillna(0)

    return df

In [12]:
df=handle_null(df)

In [13]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(['id','score'],axis=1),df['score'],test_size = 0.2,random_state=42)

In [14]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

In [14]:
# ada_best.fit(x_train_scaled, y_train)

In [15]:
from sklearn.ensemble import StackingRegressor
base_regressors = [
    ('GradientBoost', gb_best),
    ('LGBM', lgbm_best),
    ('CatBoost', cat_best),
    ('RandomForest', rforest_best),
#     ('SVR', svr_regressor)
]


stacking_regressor = StackingRegressor(estimators=base_regressors, final_estimator=LinearRegression())
stacking_regressor.fit(x_train_scaled, y_train)

stacking_predictions = stacking_regressor.predict(x_test_scaled)

stacking_rmse = mean_squared_error(y_test, stacking_predictions, squared=False)
print("Stacking Test Set RMSE:", stacking_rmse)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003209 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26485
[LightGBM] [Info] Number of data points in the train set: 1976, number of used features: 161
[LightGBM] [Info] Start training from score 3.709008
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002150 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25963
[LightGBM] [Info] Number of data points in the train set: 1580, number of used features: 160
[LightGBM] [Info] Start training from score 3.713608
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002902 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26046
[LightGBM] [Info] Number of data points in the train set: 1581, number of used features: 160
[LightGBM] [Info] Start 